In [1]:
"""
The code for patient pathogenic variant prioritization, and shap explanation plot for the pathogenic variant (Fig. 5B-D). 
Due to privacy concerns, we cannot disclose patient data.
"""

In [ ]:
import time
import matplotlib.pyplot as plt
import os
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import shap  # package used to calculate Shap values
pd.options.mode.chained_assignment = None
import pickle
import seaborn as sns

In [2]:
"---------------------------- Load Data ------------------------------"

path = os.path.join('..', '..', 'Data', 'Full_Slim_Dataset_hg37-v1.6.csv')
Slim_dataset = pd.read_csv(path, engine='python')#low_memory=False,
print(Slim_dataset)

path = os.path.join('..', '..', 'Data', 'Gene list with OMIM, ACMG, HPO_Dataset_Edited.csv')
Metadata = pd.read_csv(path)
print(Metadata)

path = os.path.join('..', '..', 'Results', 'Best_Parameters', 'Best_Parameters_New_17.csv')
Best_param = pd.read_csv(path, engine='python')#low_memory=False,
print(Best_param)

path = os.path.join('..', '..', 'Data', 'Relevant_Columns_Names_Edited_2.csv')
Relevant_Cols_df = pd.read_csv(path)
overlap_cols = Relevant_Cols_df['Feature'].tolist()
print(overlap_cols)
rename_dict = dict(zip(overlap_cols, Relevant_Cols_df['Feature Name'].tolist()))
# print(rename_dict)
overlap_cols_names  = Relevant_Cols_df['Feature Name'].tolist()



       VariationID   OMIMs                                 Manifested_Tissues  \
0           535972  613254  brain-0,kidney,Skin - Sun Exposed (Lower leg),...   
1           535875  613254  brain-0,kidney,Skin - Sun Exposed (Lower leg),...   
2           535979  613254  brain-0,kidney,Skin - Sun Exposed (Lower leg),...   
3           567376  613254  brain-0,kidney,Skin - Sun Exposed (Lower leg),...   
4           565912  613254  brain-0,kidney,Skin - Sun Exposed (Lower leg),...   
...            ...     ...                                                ...   
67963       873299     NaN                                                NaN   
67964       873211     NaN                                                NaN   
67965       873240     NaN                                                NaN   
67966       873216     NaN                                                NaN   
67967       915562     NaN                                                NaN   

      #Chr        Pos      

In [3]:
"-------------------------- Data PreProcessing ------------------------"


def preprocessing_data2(Relevant_Data, medians_dict):

    one_hot_columns = ['Type', 'AnnoType', 'Consequence', 'Domain', 'Dst2SplType']  # , 'EnsembleRegulatoryFeature'

    # Get one hot encoding of columns B
    one_hot = pd.get_dummies(Relevant_Data[one_hot_columns])
    # Drop column B as it is now encoded
    Relevant_Data = Relevant_Data.drop(one_hot_columns, axis=1)
    # Join the encoded df
    Relevant_Data = Relevant_Data.join(one_hot)
    # print(relevant_data_1)
    cHmm_columns = Slim_dataset.columns[Slim_dataset.columns.str.contains(pat='cHmm_E')].tolist()
    fill_zero_columns = ['motifECount', 'motifEHIPos', 'motifEScoreChng', 'mirSVR-Score', 'mirSVR-E', 'mirSVR-Aln',
                         'tOverlapMotifs', 'motifDist'] + cHmm_columns  # motifs with high number of nan 97%
    Relevant_Data[fill_zero_columns] = Relevant_Data[fill_zero_columns].fillna(value=0)
    fill_common_columns = ['cDNApos', 'relcDNApos', 'CDSpos', 'relCDSpos', 'protPos', 'relProtPos', 'Dst2Splice',
                           'SIFTval', 'PolyPhenVal', 'GerpRS', 'GerpRSpval', 'GerpN', 'GerpS', 'all Enc',
                           'Grantham', 'All SpliceAI', 'All MMSp', 'Dist2Mutation', 'All 00bp', 'dbscSNV',
                           'RemapOverlapTF', 'RemapOverlapCL', 'Trace Features']  # Locations, is this right?
    common_cols = [c for c in list(Relevant_Data) if c in medians_dict]
    for cl in common_cols:
        Relevant_Data[cl] = Relevant_Data[cl].fillna(medians_dict[cl])
    return Relevant_Data

def preprocessing_new(Relevant_Data):
    
    "---------------------- One Hot Columns -------------------------"
    
    one_hot_columns = ['Type', 'AnnoType', 'Consequence', 'Domain', 'Dst2SplType'] 

    one_hot = pd.get_dummies(Relevant_Data[one_hot_columns])
    Relevant_Data = Relevant_Data.drop(one_hot_columns, axis=1)
    Relevant_Data = Relevant_Data.join(one_hot)
    
    "---------------------- Missing Values Imputation ---------------"
    
    special_imputation_cols = {'SIFTval':1, 'GC':0.42, 'CpG':0.02, 'priPhCons':0.115, 'mamPhCons':0.079, 'verPhCons':0.094,'priPhyloP':-0.033, 'mamPhyloP':-0.038, 'verPhyloP':0.017, 'GerpN':1.91, 'GerpS':-0.2}
    
    for cl in special_imputation_cols:
        Relevant_Data[cl] = Relevant_Data[cl].fillna(special_imputation_cols[cl])
        
    Relevant_Data.fillna(0, inplace=True)
    
    return Relevant_Data

In [4]:
"-------------------------  SHAP ------------------------------------"

def shap_plot(j, S, model, short_name, tissue):
    explainerModel = shap.TreeExplainer(model)
    shap_values_Model = explainerModel.shap_values(S)
    print(shap_values_Model)
    print(explainerModel.expected_value)

    fig = plt.gcf()
    
    plot_title = short_name + '_' + tissue.strip()
    file_name = plot_title + '_SHAP.jpg'
    out_path = os.path.join('..', '..', 'Results', 'Patients Analysis Results New Imputation2', file_name)
    p = 0.08  # Probability 0.4
    new_base_value = np.log(p / (1 - p))  # the logit function
    shap.decision_plot(explainerModel.expected_value[1], shap_values_Model[1][j], S.iloc[[j]], show=False, highlight=0)  # Rf : explainer.expected_value[1],link='logit', new_base_value=new_base_value
    plt.title(plot_title + ' SHAP Decision Plot', x=0.5, y=1.1)
    all_axes = fig.get_axes()
    ax = all_axes[0]
    plt.tight_layout()
    plt.savefig(out_path, dpi=100)
    plt.close()


In [5]:
y_columns = Slim_dataset.columns[Slim_dataset.columns.str.contains(pat = 'disease_causing')].tolist()
print(y_columns)
non_relevant_columns = ['VariationID', 'OMIMs', 'Manifested_Tissues', '#Chr', 'Pos', 'ConsDetail', 'motifEName', 'GeneID_y', 'FeatureID', 'GeneName', 'CCDS', 'Intron', 'Exon', 'SIFTcat', 'PolyPhenCat', 'bStatistic', 'targetScan', 'dbscSNV-rf_score', 'oAA', 'Ref', 'nAA', 'Alt', 'Segway']# it will be good to replace oAA and nAA with blssuom64 matrix. What bStatistic doing?
non_relevant_columns = non_relevant_columns + y_columns
non_relevant_patient = ['#Chr', 'Pos', 'ConsDetail', 'motifEName', 'GeneID_y', 'FeatureID', 'GeneName', 'CCDS', 'Intron', 'Exon', 'SIFTcat', 'PolyPhenCat', 'bStatistic', 'targetScan', 'dbscSNV-rf_score', 'oAA', 'Ref', 'nAA', 'Alt', 'Segway']# it will be good to replace oAA and nAA with blssuom64 matrix. What bStatistic doing?

cols = Slim_dataset.columns
relevant_columns = [x for x in cols if x not in non_relevant_columns]
print(relevant_columns)
Slim_Relevant = Slim_dataset[relevant_columns]

Slim_Relevant = preprocessing_new(Slim_Relevant)

['Lung_disease_causing', 'Muscle - Skeletal_disease_causing', 'Skin - Sun Exposed_disease_causing', 'Adipose - Subcutaneous_disease_causing', 'Artery - Aorta_disease_causing', 'Heart - Left Ventricle_disease_causing', 'Artery - Coronary_disease_causing', 'brain-0_disease_causing', 'Liver_disease_causing', 'Nerve - Tibial_disease_causing', 'Colon - Sigmoid_disease_causing', 'kidney_disease_causing', 'Heart - Atrial Appendage_disease_causing', 'Breast - Mammary Tissue_disease_causing', 'Uterus_disease_causing', 'Adipose - Visceral_disease_causing', 'Esophagus - Gastroesophageal Junction_disease_causing', 'Esophagus - Mucosa_disease_causing', 'brain-1_disease_causing', 'Skin - Not Sun Exposed_disease_causing', 'Artery - Tibial_disease_causing', 'Pituitary_disease_causing', 'Ovary_disease_causing', 'brain-3_disease_causing', 'Thyroid_disease_causing', 'Testis_disease_causing', 'Whole Blood_disease_causing', 'brain-2_disease_causing', 'brain_disease_causing']
['Type', 'Length', 'AnnoType', 

In [6]:
import ast
from time import gmtime, strftime

"-------------------------------- Create Ptient Specific Model --------------------------------------------"
patient_files_list = Metadata['Dataset File'].tolist()

print(patient_files_list)
patient_files_list = [x for x in patient_files_list if pd.notnull(x)]

def patient_variants_prioritization(file):

    short_name = file.split('_')[0]
    tissues = Metadata['Tissue_Model'][Metadata['Dataset File'] == file].values[0]
    disease_gene = Metadata['Gene'][Metadata['Dataset File'] == file].values[0]
    disease_gene = disease_gene.replace("?", "")

    print('---------------------------', short_name, '------------------------')
    print('Tissues: ', tissues)

    patient_path = os.path.join('..', '..', 'Data', 'Patients_Datasets', file)
    
    Patient_data = pd.read_csv(patient_path)

    Patient_Relevant = Patient_data[relevant_columns]
    Patient_Relevant = preprocessing_new(Patient_Relevant)

    patient_genes = Patient_data['GeneID_y'].tolist()
    tissues = tissues.split(',')
    for tissue in tissues:
        print('------------------', tissue, short_name, '----------------')

        y = tissue.strip() + '_' + 'disease_causing'
        X_trian = Slim_Relevant[~Slim_dataset['GeneID_y'].isin(patient_genes)]
        y_train = Slim_dataset[y][~Slim_dataset['GeneID_y'].isin(patient_genes)]
        X_trian.rename(columns=rename_dict, inplace=True)
        Patient_Relevant.rename(columns=rename_dict, inplace=True)

        best_parameters = Best_param['Best_Parameters'][(Best_param['Dataset'] == 'Full Trace')&(Best_param['Tissue'] == tissue.strip())&(Best_param['ML_Model'] == 'Random Forest')].values[0]
        best_parameters = ast.literal_eval(best_parameters)
        print(best_parameters)
        
        model = RandomForestClassifier(**best_parameters)
        common_features = [x for x in list(Patient_Relevant) if (x in list(X_trian)) and (x in overlap_cols_names)]
        X_trian[common_features] = X_trian[common_features].fillna(0)
        model.fit(X_trian[common_features], y_train)
        Patient_Relevant[common_features] = Patient_Relevant[common_features].fillna(0)
        patient_predict_proba = model.predict_proba(Patient_Relevant[common_features])
        # print(patient_predict_proba)
        patient_predictions = patient_predict_proba[:, 1]
        prediction_df = pd.DataFrame(patient_predictions, columns=['Pathological_probability'])
        results_df = pd.concat([Patient_data, prediction_df], axis=1)
        Relevant_Results = results_df[['GeneName', 'GeneID_y', '#Chr', 'Pos', 'Ref', 'Alt', 'Type', 'Length','SIFTval', 'PolyPhenVal', 'PHRED', 'Pathological_probability']]
        Relevant_Results = Relevant_Results.sort_values('Pathological_probability', ascending=False)
        Relevant_Results['Is_pathogenic'] = False
        Relevant_Results.loc[Relevant_Results.GeneName == disease_gene, 'Is_pathogenic'] = True
        print(Relevant_Results)

        out_path = os.path.join('..', '..', 'Results', 'Patients Analysis Results New Imputation2', short_name + '_' + tissue.strip()  + '_Rank.csv')
        print('out_path', out_path)
        Relevant_Results.to_csv(out_path)

        if disease_gene in Relevant_Results['GeneName'].tolist():

            print(Relevant_Results['Pathological_probability'][Relevant_Results['GeneName'] == disease_gene])


            "Shap"
            j = Relevant_Results[Relevant_Results['Is_pathogenic'] == True].index.tolist()[0]
            print(j)
            shap_plot(j, Patient_Relevant[common_features], model, short_name, tissue)

        else:
            print('* Disease Gene: ', disease_gene, ' do not exist in prioritization results')
            
    return short_name, strftime("%Y-%m-%d %H:%M:%S", gmtime())


['AzoospermiaPMRRP28085_TRACE_CADD-v1.6_GRCh37.csv', 'AFOtB1305_TRACE_CADD-v1.6_GRCh37.csv', 'CardiacNeuralsyndromeOtA46052_TRACE_CADD-v1.6_GRCh37.csv', 'CardiomyopathyOtB0551_TRACE_CADD-v1.6_GRCh37.csv', 'HSP13393_TRACE_CADD-v1.6_GRCh37.csv', 'CardiacNeuralSyndrome41621_TRACE_CADD-v1.6_GRCh37.csv', 'CardiacNeuralSyndrome41621_TRACE_CADD-v1.6_GRCh37.csv', 'Autism13498_TRACE_CADD-v1.6_GRCh37.csv', 'Musculardystrophy13879_TRACE_CADD-v1.6_GRCh37.csv', 'Zellwegersyndrome13414_TRACE_CADD-v1.6_GRCh37.csv', 'SeckelSyndromeSS2_TRACE_CADD-v1.6_GRCh37.csv', 'Leighsyndrome766M_TRACE_CADD-v1.6_GRCh37.csv', 'HSP16390_TRACE_CADD-v1.6_GRCh37.csv', 'Skin16264_TRACE_CADD-v1.6_GRCh37.csv', 'MicrocephalyOt1097_TRACE_CADD-v1.6_GRCh37.csv', 'PCCAOt1103_TRACE_CADD-v1.6_GRCh37.csv', 'PCCAOt1103_TRACE_CADD-v1.6_GRCh37.csv', 'PCCAOt1104_TRACE_CADD-v1.6_GRCh37.csv', 'PCCAOt1104_TRACE_CADD-v1.6_GRCh37.csv', 'NPHPOt1114_TRACE_CADD-v1.6_GRCh37.csv', 'HypotoniaOt2812_TRACE_CADD-v1.6_GRCh37.csv', 'diaphragmaticherni

In [7]:
import multiprocessing as mp

def driver_func_shap():
    PROCESSES = 40
    df_list = []
    
    with mp.Pool(PROCESSES) as pool:
        results = [pool.apply_async(patient_variants_prioritization(file, )) for file in patient_files_list]
        for r in results:
            results_tuple = r.get(timeout=None)
            print('@', results_tuple[0], ' finished', results_tuple[1])

            
if __name__ == '__main__':
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))
    driver_func_shap()    

2022-12-04 10:49:58
--------------------------- AzoospermiaPMRRP28085 ------------------------
Tissues:  brain, Testis
------------------ brain AzoospermiaPMRRP28085 ----------------
{'random_state': 1234, 'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_depth': 40, 'bootstrap': False}
     GeneName         GeneID_y #Chr        Pos  \
97     SCAPER  ENSG00000140386   15   76866531   
124     CEP89  ENSG00000121289   19   33444610   
70     NDUFS3  ENSG00000213619   11   47593021   
49   KIAA1549  ENSG00000122778    7  138554407   
133   ARFGAP1  ENSG00000101199   20   61919110   
..        ...              ...  ...        ...   
16      DNAH6  ENSG00000115423    2   84806635   
137     BEND2  ENSG00000177324    X   18221974   
8        CR1L  ENSG00000197721    1  207850792   
63       EXD3  ENSG00000187609    9  140287591   
146     TBX22  ENSG00000122145    X   79283556   

                                                   Ref Alt Type  Length  \
97         

--------------------------- AFOtB1305 ------------------------
Tissues:  Heart - Left Ventricle
------------------ Heart - Left Ventricle AFOtB1305 ----------------


Columns (0,13,113) have mixed types.Specify dtype option on import or set low_memory=False.


{'random_state': 1234, 'n_estimators': 600, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': None, 'bootstrap': False}
    GeneName         GeneID_y #Chr        Pos  Ref Alt Type  Length  SIFTval  \
95     XIRP1  ENSG00000168334    3   39227886  ATG   A  DEL       2      NaN   
416     ENO3  ENSG00000108515   17    4859921    G   A  SNV       0     0.00   
187     TRDN  ENSG00000186439    6  123696752    G   A  SNV       0     0.00   
489     BMP7  ENSG00000101144   20   55777618    C   T  SNV       0     0.01   
44     OBSCN  ENSG00000154358    1  228538629    C   T  SNV       0     0.00   
..       ...              ...  ...        ...  ...  ..  ...     ...      ...   
429     CDC6  ENSG00000094804   17   38445837    C   T  SNV       0      NaN   
252   RNF208  ENSG00000212864    9  140115273    G   A  SNV       0     0.26   
325    RPAP3  ENSG00000005175   12   48063983    T   G  SNV       0     0.18   
262   ENTPD1  ENSG00000138185   10   97605282    G   A  SNV       0   

[array([[ 5.44804562e-04, -3.18449113e-03,  1.71331798e-06, ...,
         6.41864409e-06, -2.74309292e-05,  4.33326634e-06],
       [-9.14160416e-04, -1.55561740e-03,  2.57657854e-05, ...,
         1.08403628e-05,  2.00126097e-06,  1.30006085e-06],
       [ 2.86186673e-04, -2.70486057e-03,  2.79719987e-05, ...,
         2.67711129e-06,  4.33967934e-06,  2.99152382e-06],
       ...,
       [-1.60926758e-03, -2.29677962e-03,  7.32214417e-05, ...,
         6.38134600e-06,  4.68308900e-06,  1.86390827e-06],
       [ 2.44402024e-04, -1.72804496e-03,  7.61325620e-06, ...,
         8.54784507e-06,  3.31090456e-06,  2.36576531e-06],
       [ 2.47455158e-04, -2.98892396e-03,  9.23728593e-05, ...,
         7.95307015e-06,  3.95128387e-06,  3.78760429e-07]]), array([[-5.44804562e-04,  3.18449113e-03, -1.71331798e-06, ...,
        -6.41864409e-06,  2.74309292e-05, -4.33326634e-06],
       [ 9.14160416e-04,  1.55561740e-03, -2.57657854e-05, ...,
        -1.08403628e-05, -2.00126097e-06, -1.30006085

...

Columns (0,113) have mixed types.Specify dtype option on import or set low_memory=False.


{'random_state': 1234, 'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_depth': 40, 'bootstrap': False}
    GeneName         GeneID_y #Chr        Pos Ref       Alt Type  Length  \
398   IQSEC3  ENSG00000120645   12     250288   A         C  SNV       0   
242     FIG4  ENSG00000112367    6  110053838   A         T  SNV       0   
601      CIC  ENSG00000079432   19   42799294   A        AG  INS       1   
269    CDK13  ENSG00000065883    7   40127724   G         T  SNV       0   
123   PTPN23  ENSG00000076201    3   47454409   A  ACCCCCCC  INS       7   
..       ...              ...  ...        ...  ..       ...  ...     ...   
310  POU5F1B  ENSG00000212993    8  128428322   G         C  SNV       0   
315   IFNA10  ENSG00000186803    9   21206532   C         T  SNV       0   
551    TEX14  ENSG00000121101   17   56676457   T         C  SNV       0   
363    MUC5B  ENSG00000117983   11    1261508   C         G  SNV       0   
653   PIWIL3  ENSG00000184571   22

[array([[-2.30619895e-03, -2.38251477e-03,  1.56023246e-05, ...,
         5.49535190e-06,  2.75377968e-06,  4.46560772e-06],
       [ 2.23034516e-04, -1.44812225e-03,  8.40784614e-05, ...,
         2.44025258e-06,  6.31253468e-06,  1.27797953e-05],
       [-4.54058001e-03,  6.72421699e-03,  4.58167828e-05, ...,
         7.64780810e-06,  6.25127701e-06,  9.29353843e-06],
       ...,
       [-1.59304973e-03, -1.40330409e-03,  3.31055879e-05, ...,
         4.71791921e-06,  3.32915506e-06,  4.37069931e-06],
       [ 3.27195037e-04, -1.87101774e-03,  1.08883587e-05, ...,
         1.06975770e-06,  3.91689807e-06,  6.31555038e-06],
       [ 4.22592045e-04, -2.22857700e-03,  4.55385471e-07, ...,
         8.66457838e-07,  4.50377371e-06, -2.65278905e-05]]), array([[ 2.30619895e-03,  2.38251477e-03, -1.56023246e-05, ...,
        -5.49535190e-06, -2.75377968e-06, -4.46560772e-06],
       [-2.23034516e-04,  1.44812225e-03, -8.40784615e-05, ...,
        -2.44025258e-06, -6.31253468e-06, -1.27797953

--------------------------- Usher16032 ------------------------
Tissues:  brain
------------------ brain Usher16032 ----------------
{'random_state': 1234, 'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_depth': 40, 'bootstrap': False}
     GeneName         GeneID_y #Chr        Pos         Ref  \
43      GPR98  ENSG00000164199    5   90059126          AT   
19      PRKRA  ENSG00000180228    2  179312231           A   
28       CHL1  ENSG00000134121    3     432737           C   
33   KIAA2018  ENSG00000176542    3  113376117  ATGCTGCTGC   
138     HAUS5  ENSG00000249115   19   36109302           A   
..        ...              ...  ...        ...         ...   
91     FBRSL1  ENSG00000112787   12  133086349           G   
137     HAUS5  ENSG00000249115   19   36109297           A   
139     PSMD8  ENSG00000099341   19   38866785           T   
71   PTCHD3P1  ENSG00000224597   10   29770662           A   
74       SPRN  ENSG00000203772   10  135381814         

TypeError: 'tuple' object is not callable